In [1]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from keras.models import Model, load_model
from keras.layers import Input, Flatten, Dense, concatenate#,  Dropout
from keras.optimizers import Adam
#from keras.utils import plot_model
#from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
#from sklearn.decomposition import PCA
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from keras import regularizers
from keras import backend as K
#import cv2
import os
from glob import glob
#import tqdm
from PIL import Image
#import scipy.io as sio
import xgboost as xgb
#from xgboost import plot_importance
import time
#import pickle
import keras2onnx
import onnx
import random
#import gc
from keras.callbacks import Callback
#from keras.callbacks import EarlyStopping
#from tensorflow.python.keras.callbacks import ModelCheckpoint
import argparse

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import math
    
def get_data_num_floor_function(split_dir, total_class_number):
    list_of_img_num_in_each_class = []
    
    for i in range(0, total_class_number):
        file_list = os.listdir(split_dir+str(i)) # file_dir is your directory path
        img_num_in_each_class = len(file_list)
        list_of_img_num_in_each_class.append(img_num_in_each_class)
        
    the_min_img_num = min(list_of_img_num_in_each_class)
    int_value_of_negative_classes = math.floor(the_min_img_num/5)
    int_value_of_positive_class = int_value_of_negative_classes*5
    
    return int_value_of_negative_classes, int_value_of_positive_class

In [4]:
# python trTripletOstIter2.py --mN 1
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--mN", type=int)
#args = vars(ap.parse_args())

_StoreAction(option_strings=['-m', '--mN'], dest='mN', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help=None, metavar=None)

In [5]:
picSize=224

In [6]:
def get_val_data(val_dir, dirName, total_class_number, positive_class_index):
    negative_img_num, positive_img_num = get_data_num_floor_function(val_dir, total_class_number)
    data = glob(os.path.join(val_dir+str(dirName)+"/", "*.jpg"))
    data = np.sort(data)
    st1=0
    if dirName == positive_class_index:
        nd1 = positive_img_num
    else:
        nd1 = negative_img_num
    #nd1=data.size
    x1 = np.zeros((len(range(st1,nd1,1)), picSize, picSize, 1),dtype='float16')
    y1 = np.ones((len(range(st1,nd1,1)),),dtype='uint8')*dirName
    k=-1
    li1 = [i for i in range(data.size)]
    j=random.sample(li1,nd1)
    dataName=[]
    for i in range(st1, nd1, 1):
        image = Image.open(data[j[i]])
        dataName.append(data[j[i]])
        image = np.asarray(image) / 255.0   
        #a=image[:,:,1]
        a=image
        b=a.reshape((picSize,picSize,1))
        k=k+1
        x1[k] = b
    x_data=x1
    y_data=y1
    return x_data.astype('float16'),y_data.astype('uint8'),dataName

In [7]:
def get_tr_data(train_dir, dirName, total_class_number, positive_class_index): 
    negative_img_num, positive_img_num = get_data_num_floor_function(train_dir, total_class_number)
    data = glob(os.path.join(train_dir+str(dirName)+"/", "*.jpg"))
    data = np.sort(data)
    st0=0
    if dirName == positive_class_index:
        nd0 = positive_img_num
    else:
        nd0 = negative_img_num
    #nd0=data.size
    x0 = np.zeros((len(range(st0,nd0,1)), picSize, picSize, 1),dtype='float16')
    y0 = np.ones((len(range(st0,nd0,1)),),dtype='uint8')*dirName
    k=-1
    li0 = [i for i in range(data.size)]
    j=random.sample(li0, nd0)    
    for i in range(st0, nd0, 1):
        image = Image.open(data[j[i]]) 
        image = np.asarray(image) / 255.0
        #a=image[:,:,1]
        a=image
        b=a.reshape((picSize,picSize,1))
        k=k+1
        x0[k] = b    
    return x0.astype('float16'),y0.astype('uint8')

In [8]:
def categorical_squared_hinge(y_true, y_pred):
    y_true = 2. * y_true - 1
    vvvv = K.maximum(1. - y_true * y_pred, 0.)
    vv = K.sum(vvvv, 1, keepdims=False)
    v = K.mean(vv, axis=-1)
    return v

In [9]:
def pairwise_distance(feature, squared=False):
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))
    num_data = array_ops.shape(feature)[0]
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

In [10]:
def masked_maximum(data, mask, dim=1):
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

In [11]:
def triplet_loss_adapted_from_tf(y_true, y_pred):
    del y_true
    margin = 1.
    labels = y_pred[:, :1] 
    labels = tf.cast(labels, dtype='int32')
    embeddings = y_pred[:, 1:]
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    adjacency_not = math_ops.logical_not(adjacency)
    batch_size = array_ops.size(labels)
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)
    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)
    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)
    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))
    num_positives = math_ops.reduce_sum(mask_positives)
    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    return semi_hard_triplet_loss_distance

In [12]:
def create_base_network2(image_input_shape, embedding_size):   
    model = Sequential()
    inputShape = (picSize, picSize, 1)
    chanDim = -1
    model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    model.add(Flatten())    
    model.add(Dense(embedding_size))
    base_network=model
    return base_network

In [13]:
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='loss', value=0.001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)
        else:
            if current < self.value:
                if self.verbose > 0.0:
                    print("Epoch %05d: early stopping THR" % epoch)
                self.model.stop_training = True

In [14]:
starttime = time.localtime(time.time())
input_image_shape = (picSize, picSize, 1)
embedding_size = 128
opt = Adam(lr=1e-6)

In [15]:
train_dir = '/raid/home/ben/_kunyi/triplet_net/tea_3/tea_image_datasets/13_tea_classes_splittedImgs/13_tea_classes_V4_iphoneFormat_onlyOolongTea_trainValTest/train/'
val_dir = '/raid/home/ben/_kunyi/triplet_net/tea_3/tea_image_datasets/13_tea_classes_splittedImgs/13_tea_classes_V4_iphoneFormat_onlyOolongTea_trainValTest/val/'

In [16]:
positive_class_index = 0
classN = 6
dirName = 0

In [17]:
## training data
x_train, y_train = get_tr_data(train_dir, dirName, classN, positive_class_index)
for i in range(1,classN,1):
    xi,yi = get_tr_data(train_dir, i, classN, positive_class_index)
    x_train=np.concatenate((x_train,xi), axis=0)
    y_train=np.concatenate((y_train,yi), axis=0)

In [18]:
## validation data
x_test, y_test, dN = get_val_data(val_dir, dirName, classN, positive_class_index)
for i in range(1,classN,1):
    xi,yi,dNi = get_val_data(val_dir, i, classN, positive_class_index)    
    dN=np.concatenate((dN,dNi), axis=0)
    x_test=np.concatenate((x_test,xi), axis=0)
    y_test=np.concatenate((y_test,yi), axis=0)

In [19]:
mN = 0 

In [20]:
if (mN==0): # 建新模型 modelN=0
    base_network = create_base_network2(input_image_shape, embedding_size)
    input_images = Input(shape=input_image_shape, name='input_image')
    input_labels = Input(shape=(1,), name='input_label')
    embeddings = base_network([input_images])
    labels_plus_embeddings = concatenate([input_labels, embeddings])
    model = Model(inputs=[input_images, input_labels],outputs=labels_plus_embeddings)

In [21]:
if (mN>0): # 疊代使用 modelN=1,2,3,...
    model = load_model("trip_Ost_"+str(mN-1)+".h5", 
                       custom_objects={'triplet_loss_adapted_from_tf':triplet_loss_adapted_from_tf})
    testing_embeddings = create_base_network2(input_image_shape,embedding_size=embedding_size)
    for layer_target, layer_source in zip(testing_embeddings.layers, model.layers[2].layers):
        weights = layer_source.get_weights()
        layer_target.set_weights(weights)
        del weights

In [22]:
model.compile(loss=triplet_loss_adapted_from_tf, optimizer=opt)


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
callbacks_list = [EarlyStoppingByLossVal()]
dummy_gt_train = np.zeros((len(x_train), embedding_size + 1)) 
dummy_gt_val = np.zeros((len(x_test), embedding_size + 1))
x_train = np.reshape(x_train, (len(x_train), x_train.shape[1], x_train.shape[1], 1))
x_test = np.reshape(x_test, (len(x_test), x_train.shape[1], x_train.shape[1], 1))
#x_train = np.reshape(x_train, (len(x_train), x_train.shape[0], x_train.shape[1], 1))

In [24]:
start = time.perf_counter()

In [25]:
batch_size = 128
epochs = 1000

In [ ]:
train_history = model.fit(
    x=[x_train,y_train],
    y=dummy_gt_train,    
    batch_size=batch_size,
    epochs=epochs, 
    validation_data=([x_test, y_test], dummy_gt_val),
    callbacks=callbacks_list)



Train on 290400 samples, validate on 36300 samples
Epoch 1/1000
290400/290400 [==============================] - 499s 2ms/step - loss: 1.3410 - val_loss: 1027.6321
Epoch 2/1000
290400/290400 [==============================] - 488s 2ms/step - loss: 1.0237 - val_loss: 1026.7143
Epoch 3/1000
290400/290400 [==============================] - 489s 2ms/step - loss: 0.6485 - val_loss: 939.4409
Epoch 4/1000
290400/290400 [==============================] - 489s 2ms/step - loss: 0.4635 - val_loss: 947.7999
Epoch 5/1000
290400/290400 [==============================] - 488s 2ms/step - loss: 0.3396 - val_loss: 950.0628
Epoch 6/1000
290400/290400 [==============================] - 488s 2ms/step - loss: 0.2446 - val_loss: 941.0029
Epoch 7/1000
290400/290400 [==============================] - 488s 2ms/step - loss: 0.1905 - val_loss: 933.8397
Epoch 8/1000
290400/290400 [==============================] - 488s 2ms/step - loss: 0.1484 - val_loss: 924.5859
Epoch 9/1000
290400/290400 [=====================

290400/290400 [==============================] - 488s 2ms/step - loss: 0.0056 - val_loss: 585.6789
Epoch 142/1000
137728/290400 [=============>................] - ETA: 4:04 - loss: 0.0056

In [ ]:
time_taken = time.perf_counter() - start
print("Need ", time_taken, "seconds to train a model, which is")
print(time_taken/3600, "hours")

In [ ]:
model.save("trip_Ost_"+str(mN)+".h5")

In [ ]:
%matplotlib inline
plt.plot(train_history.history['loss'], label='train loss')
plt.plot(train_history.history['val_loss'], label='validation loss')
plt.legend()
plt.title('Train/Validation loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
testing_embeddings = create_base_network2(input_image_shape,embedding_size=embedding_size)
#testing_embeddings = create_base_network3(input_image_shape,embedding_size=embedding_size)

In [ ]:
for layer_target, layer_source in zip(testing_embeddings.layers, model.layers[2].layers):
    weights = layer_source.get_weights()
    layer_target.set_weights(weights)
    del weights

In [ ]:
testing_embeddings.save("tripOst_"+str(mN)+".h5")    
model2 = load_model("tripOst_"+str(mN)+".h5")

In [ ]:
#onnx_model = keras2onnx.convert_keras(model2, model2.name,target_opset=7)
#onnx.save_model(onnx_model, "Trmodel-2_"+str(mN)+".onnx")

In [ ]:
x_train_embeddings = testing_embeddings.predict(np.reshape(x_train, (len(x_train), picSize, picSize, 1)))
x_train_emb=np.array(x_train_embeddings)
dtrain = xgb.DMatrix(x_train_emb, np.array(y_train))

In [ ]:
x_test_embeddings = testing_embeddings.predict(np.reshape(x_test, (len(x_test), picSize, picSize, 1)))
x_test_emb=np.array(x_test_embeddings)
dtest = xgb.DMatrix(x_test_emb, np.array(y_test))

In [ ]:
params ={'eta': 0.1,
      'max_depth': 20, #5 predict ability is related to max_depth
      'num_boost_round':20,
      'objective': 'multi:softmax',
      'random_state': 27,
      'num_class':classN}

In [ ]:
watchlist = [(dtrain, 'train'), (dtest, 'valid')]
model4 = xgb.train(params, dtrain, 1000, watchlist, early_stopping_rounds=1000, verbose_eval=100)

In [ ]:
model4.save_model("Trmodel_"+str(mN)+".model")  
model4.dump_model("Trmodel_"+'xgb.'+str(mN)+'.model.dump', fmap='', with_stats=True)

In [ ]:
preds_tr = model4.predict(xgb.DMatrix(x_train_emb))

In [ ]:
pred_sc_tr=accuracy_score(preds_tr, np.array(y_train))
print("Training accuracy = ", pred_sc_tr)

In [ ]:
preds_test = model4.predict(xgb.DMatrix(x_test_emb))

In [ ]:
pred_sc_test=accuracy_score(preds_test, np.array(y_test))
print("Testing accuracy = ",pred_sc_test)

In [ ]:
endtime = time.localtime(time.time())
print("starttime:", starttime) 
print("endtime:", endtime)